In [1]:
# --------------------------------------------------------
# adaptd from the demo (RA)
# Fully Convolutional Instance-aware Semantic Segmentation
# Copyright (c) 2017 Microsoft
# Licensed under The MIT License [see LICENSE for details]
# Written by Haochen Zhang, Yi Li, Haozhi Qi
# --------------------------------------------------------
import pandas as pd
import os
import sys
os.chdir("../FCIS/fcis") # added because this script expects to be run where demo.py lcoated in fcis repo
import _init_paths
import utils.image as image
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec
import random
import cv2
import numpy as np
from dataset.pycocotools.coco import COCO
from dataset.pycocotools import mask as maskUtils
compare_py_path = "../../code/utils/"
sys.path.append(os.path.abspath(compare_py_path))
import compare
from config.config import config, update_config
update_config('../../code/model/resnet_v1_101_coco_fcis_end2end_ohem-nebraska-128-moresamples.yaml') # pointing to nebraska model config
validation_imagesp = os.path.join(config['dataset']['dataset_path'], "images", "val-nebraska")
test_imagesp = os.path.join(config['dataset']['dataset_path'],  "images", "test-nebraska")
output_fig_dir_validation = os.path.join(config['dataset']['root_path'], "output/figs/validation")
output_fig_dir_test= os.path.join(config['dataset']['root_path'], "output/figs/test")

# deterimes val or test run
anns_path = "/home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/annotations/instances_test-nebraska.json"
image_dir = test_imagesp
output_fig_dir = output_fig_dir_test

coco = COCO(anns_path) # need to load image paths from annotations file so that order of iamges matches order of annotations
image_names = [os.path.join(image_dir, subdict['file_name']) for key, subdict in coco.imgs.items()]

config/config.py:173: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))


loading annotations into memory...
Done (t=0.68s)
creating index...
index created!


# Evaluating Validation Set (can't evaluate test set on the fly, but the validation set wasn't used to change any hyperparameters)

In [2]:
from dataset.pycocotools.coco import COCO
from dataset.pycocotools.cocoeval import COCOeval

cocoGt=COCO("/home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/annotations/instances_val-nebraska.json")
cocoDt=cocoGt.loadRes("/home/data/output/resnet_v1_101_coco_fcis_end2end_ohem-nebraska-128-moresamples/val-nebraska/results/detections_val-nebraska_results.json")

cocoEval = COCOeval(cocoGt,cocoDt)
cocoEval.params.areaRng = [[0 ** 2, 1e5 ** 2], [0,21.805885 ** 2], [21.805885 ** 2, 24.016960 ** 2], [24.016960 ** 2, 1e5 ** 2]]
cocoEval.params.maxDets = [10, 25, 50]
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=2.20s)
creating index...
index created!
Loading and preparing results...     
DONE (t=2.73s)
creating index...
index created!
Running per image evaluation...      
DONE (t=32.98s).
Accumulating evaluation results...   
DONE (t=1.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.782
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.671
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.764
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.693
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.620
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.632
 A

# Making figures for All Validation Images

In [2]:
plt.ioff()
all_maps = []
all_mars = []
all_ims, all_dets, all_masks, all_configs, all_classes = compare.predict_on_image_names(image_names, config, model_path_id="/home/data/output/resnet_v1_101_coco_fcis_end2end_ohem-nebraska-128-moresamples/train-nebraska/e2e", epoch=1)
for index in range(len(all_ims)):
    coco_anns = compare.load_coco_annotation(index+1, anns_path,  image_dir)
    if coco_anns['boxes'].shape[0] == 0 and all_masks[index][0].shape[0] != 0:
        all_maps.append(0) # no true positives or false positves so we set precision to 0
        all_mars.append(0) # no true positives, so recall is 0
        pred_boxes, pred_class_ids, pred_boxes, maskspred = compare.munge_p(coco_anns, index, all_ims, all_dets, all_masks)
        plt.style.use("seaborn")
        fig, ax = plt.subplots(1,2, figsize=(12,12))
        height, width = all_ims[index-1].shape[:2]
        ax[1].set_ylim(height + 10, -10)
        ax[1].set_xlim(-10, width + 10)
        ax[1].axis('off')
        ax[1].imshow(all_ims[index])
        ax[1].set_title("2005 Groundtruth Labels (No Labels)", fontsize=18)
        compare.display_instances(all_ims[index], maskspred, pred_boxes, ax=ax[0], ec= "green", title="FCIS Model Predictions")
        plt.tight_layout()
        
    elif coco_anns['boxes'].shape[0] == 0 and all_masks[index][0].shape[0] == 0:
        all_maps.append(1) # no false positves so we set precision to 1
        all_mars.append(1) # no false negatives, so recall is 1
        
    elif all_masks[index][0].shape[0] != 0:
        gt_boxes, gt_class_ids, masksgt, pred_boxes, pred_class_ids, pred_scores, maskspred, positive_ids = compare.munge_predictions_gt(index, all_ims, all_dets, all_masks, coco_anns)
        AP_text, AR_text, aps, ars  = compare.calc_stats_and_strs(gt_boxes, gt_class_ids, masksgt, pred_boxes, pred_class_ids, 
                                     pred_scores, maskspred)
        statsdf, map_s, mar_s = compare.stats_df(aps, ars)
        all_maps.append(map_s)
        all_mars.append(mar_s)
        plt.style.use("seaborn")
        gs = gridspec.GridSpec(2, 2)
        mpl.rcParams['font.family'] = 'STIXGeneral'
        plt.rcParams['font.size'] = 20
        plt.rcParams['axes.linewidth'] = 2
        plt.style.use("seaborn")
        fig = plt.figure(figsize=(10,10))
        ax0 = pl.subplot(gs[0,0])
        ax1 = pl.subplot(gs[0,1])
        ax2 = pl.subplot(gs[1, :])
        compare.display_instances(all_ims[index], maskspred, pred_boxes, ax=ax0, title="FCIS Model Predictions", ec= "purple")
        compare.display_instances(all_ims[index], masksgt, gt_boxes, ax=ax1, ec= "green", title="2005 Groundtruth Labels", detection_mask=False)
        scores = pd.DataFrame(pred_scores, columns = ["Detection Likelihood Scores"])
        boxprops = dict(linestyle='-', linewidth=4, color='k')
        medianprops = dict(linestyle='-', linewidth=4, color='k')
        scores.boxplot(ax =ax2, vert=False, fontsize=14,  showmeans=True,
                    boxprops=boxprops,
                    medianprops=medianprops)
        y_axis = ax2.axes.get_yaxis()
        y_axis.set_visible(False)
        ax2.set_title("Detection Likelihood Scores", fontsize=20)
        plt.tight_layout()
    else:
        all_maps.append(0) # no true positives or false positves so we set precision to 0
        all_mars.append(0) # no true positives, so recall is 0
        gt_class_ids, masksgt = compare.munge_g(coco_anns, index, all_ims, all_dets, all_masks)
        plt.style.use("seaborn")
        fig, ax = plt.subplots(1,2, figsize=(12,12))
        height, width = all_ims[index-1].shape[:2]
        ax[0].set_ylim(height + 10, -10)
        ax[0].set_xlim(-10, width + 10)
        ax[0].axis('off')
        ax[0].imshow(all_ims[index])
        ax[0].set_title("FCIS Model Predictions (None Predicted)", fontsize=18)
        compare.display_instances(all_ims[index], masksgt, coco_anns['boxes'], ax=ax[1], ec= "green", title="2005 Groundtruth Labels", detection_mask=False)
        plt.tight_layout()
    plt.savefig(os.path.join(output_fig_dir, str(index)+"test.png"))
    # "mAP 0.5-0.95: "+str(map_s), "mAR 0.5-0.95: "+str(map_r)

('use mxnet at', '/home/mxnet/python/mxnet/__init__.pyc')
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000001.jpg 0.0665s
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000002.jpg 0.0558s
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000003.jpg 0.0552s
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000004.jpg 0.0543s
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000005.jpg 0.0549s
(128, 128)
testing /home/data/preprocessed/test-ard-june-sept-rgb-jpeg-split-geo-128/images/test-nebraska/COCO_test-nebraska_000000000006.jpg 0.0552s
(128, 128)
testing /home/data/preprocessed

/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
AP at IOU 0.50: 0.020
AP at IOU 0.55: 0.020
AP at IOU 0.60: 0.020
AP at IOU 0.65: 0.020
AP at IOU 0.70: 0.020
AP at IOU 0.75: 0.020
AP at IOU 0.80: 0.020
AP at IOU 0.85: 0.020
AP at IOU 0.90: 0.020
AP at IOU 0.95: 0.000
AP across IOUs 0.50-0.95: 0.018
AR at IOU 0.50: 0.800
AR at IOU 0.55: 0.800
AR at IOU 0.60: 0.760
AR at IOU 0.65: 0.720
AR at IOU 0.70: 0.680
AR at IOU 0.75: 0.600
AR at IOU 0.80: 0.360
AR at IOU 0.85: 0.160
AR at IOU 0.90: 0.040
AR at IOU 0.95: 0.000
AR across IOUs 0.50-0.95: 0.492
loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
AP at IOU 0.50: 0.032
AP at IOU 0.55: 0.032
AP at IOU 0.60: 0.032
AP at IOU 0.65: 0.032
AP at IOU 0.70: 0.032
AP at IOU 0.75: 0.032
AP at IOU 0.80: 0.032
AP at IOU 0.85: 0.032
AP at IOU 0.90: 0.032
AP at IOU 0.95: 0.000
AP across IOUs 0.50-0.95: 0.029
AR at IOU 0.50: 0.720
AR at IOU 0.55: 0.720
AR at IOU 0.60: 0.680
AR at IOU 0.

KeyboardInterrupt: 

In [ ]:
np.mean(all_maps)

In [ ]:
np.mean(all_mars)

In [4]:
import json

with open("/home/data/output/resnet_v1_101_coco_fcis_end2end_ohem-nebraska/val-nebraska/results/detections_val-nebraska_results.json", "r") as f:
    data = json.load(f)

In [6]:
data[-1]

{u'category_id': 1,
 u'image_id': 42,
 u'score': 0.3882685601711273,
 u'segmentation': {u'counts': u'RjZ5:f?0O1N5M0000000000000000000000000002N00oU[2',
  u'size': [512, 512]}}

In [7]:
import pickle
with open('/home/data/output/resnet_v1_101_coco_fcis_end2end_ohem-nebraska/val-nebraska/COCO_val-nebraska_masks.pkl', 'rb') as f:
    data = pickle.load(f)

In [16]:
data[1][0].shape

(100, 21, 21)